In [1]:
!pip install -qU fal-client langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 69.2 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata

os.environ["FAL_KEY"] = userdata.get('FAL_KEY')
os.environ["OPENROUTER_API_KEY"] = userdata.get('OPENROUTER_API_KEY')

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

def generate_podcast_transcript(topic):
    podcast_template = ChatPromptTemplate.from_template("""
      Create an engaging conversation between two speakers discussing the topic: {topic}

      Requirements:
      - Generate exactly 5 back-and-forth exchanges
      - Make it natural and conversational
      - Include specific details about the {topic}
      - Each line should start with either "Speaker 1:" or "Speaker 2:"

      Here's an example of the format (but create NEW content about {topic}, don't copy this example):
      Speaker 1: [First speaker's line]
      Speaker 2: [Second speaker's line]

      The response of the each speaker should be at most 20 words. The conversation has to be insightful, engaging, explanatory, deep diving and educational.

      It should be in the style of a podcast where one speaker slightly is more knowledgeable than the other.

      You are allowed to write only in the below format. Just give the output in the below format in a single string. No additional delimiters.

      The content should be explanatory, deep diving and educational.

      Speaker 1: Hey, did you catch the game last night?
      Speaker 2: Of course! What a match—it had me on the edge of my seat.
      Speaker 1: Same here! That last-minute goal was unreal. Who's your MVP?
      Speaker 2: Gotta be the goalie. Those saves were unbelievable.


      Remember: Create completely new dialogue about {topic}, don't use the above example.
      """)

    # Initialize the ChatOpenAI model
    llm = ChatOpenAI(
        model="deepseek/deepseek-chat",
        openai_api_key=os.getenv("OPENROUTER_API_KEY"),
        openai_api_base="https://openrouter.ai/api/v1"
    )

    # Create the chain
    chain = podcast_template | llm

    response = chain.invoke({"topic": topic})
    return response.content


In [4]:
response = generate_podcast_transcript("Quantum Random Walks")
print(response)

Speaker 1: Ever heard of Quantum Random Walks? They’re like classical walks but with quantum mechanics twists.  
Speaker 2: Yeah, but how do they differ? Superposition, right?  
Speaker 1: Exactly! Unlike classical walks, particles exist in multiple states simultaneously.  
Speaker 2: So, it’s not just left or right—it’s both? That’s wild.  
Speaker 1: Precisely! And this leads to faster exploration of possibilities.  
Speaker 2: Sounds like it could revolutionize algorithms. Any real-world applications yet?  
Speaker 1: Definitely! They’re used in quantum search algorithms and simulating complex systems.  
Speaker 2: Wow, so they’re not just theoretical. That’s mind-blowing.  
Speaker 1: Yep, the quantum world is full of surprises and potential!  
Speaker 2: I’m hooked. Quantum Random Walks are next-level fascinating!


In [5]:
def generate_podcast(topic):
    print(f"\n🎙️ Generating podcast transcript about: {topic}")
    print("-" * 50)

    # Get transcript first using generate_podcast_transcript
    transcript_result = generate_podcast_transcript(topic)

    print("\n✍️ Generated transcript:")
    print("-" * 50)
    print(transcript_result)

    print("\n🔊 Converting transcript to audio...")
    print("-" * 50)

    # Progress callback for fal-client
    def on_queue_update(update):
        if isinstance(update, fal_client.InProgress):
            for log in update.logs:
                print(f"🎵 {log['message']}")

    # Generate audio using fal-client
    try:
        result = fal_client.subscribe(
            "fal-ai/playht/tts/ldm",
            {
                "input": transcript_result,
                "voices": [
                    {
                        "voice": "Jennifer (English (US)/American)",
                        "turn_prefix": "Speaker 1: "
                    },
                    {
                        "voice": "Dexter (English (US)/American)",
                        "turn_prefix": "Speaker 2: "
                    }
                ]
            },
            with_logs=True,
            on_queue_update=on_queue_update,
        )

        print("\n✅ Audio generation complete!")
        print(f"🔗 Audio URL: {result['audio']['url']}")

        return {
            "conversation": transcript_result,
            "audio_url": result['audio']['url']
        }

    except Exception as e:
        print(f"\n❌ Error generating audio: {str(e)}")
        return {
            "conversation": transcript_result,
            "audio_url": None,
            "error": str(e)
        }


In [6]:
generate_podcast("Quantum Random Walks")


🎙️ Generating podcast transcript about: Quantum Random Walks
--------------------------------------------------

✍️ Generated transcript:
--------------------------------------------------
Speaker 1: Have you heard about quantum random walks? They’re fascinating!  
Speaker 2: Vaguely. Isn’t it like classical random walks but quantum?  
Speaker 1: Exactly! But with superposition and interference—way more complex and efficient.  
Speaker 2: How does superposition affect the walker’s movement?  
Speaker 1: The walker explores multiple paths simultaneously, leading to faster exploration of graphs.  
Speaker 2: That’s mind-blowing! So, quantum walks have applications in algorithms, right?  
Speaker 1: Yes! They’re used in quantum search and even simulating quantum systems.  
Speaker 2: Impressive! I need to dive deeper into this quantum goldmine.  
Speaker 1: Start with discrete-time quantum walks—they’re simpler but still incredibly powerful.  
Speaker 2: Will do! This could revolutionize

{'conversation': 'Speaker 1: Have you heard about quantum random walks? They’re fascinating!  \nSpeaker 2: Vaguely. Isn’t it like classical random walks but quantum?  \nSpeaker 1: Exactly! But with superposition and interference—way more complex and efficient.  \nSpeaker 2: How does superposition affect the walker’s movement?  \nSpeaker 1: The walker explores multiple paths simultaneously, leading to faster exploration of graphs.  \nSpeaker 2: That’s mind-blowing! So, quantum walks have applications in algorithms, right?  \nSpeaker 1: Yes! They’re used in quantum search and even simulating quantum systems.  \nSpeaker 2: Impressive! I need to dive deeper into this quantum goldmine.  \nSpeaker 1: Start with discrete-time quantum walks—they’re simpler but still incredibly powerful.  \nSpeaker 2: Will do! This could revolutionize computing as we know it.',
 'audio_url': None,
 'error': "name 'fal_client' is not defined"}